# Introduction

In [1]:
from ipywidgets import interact
from upsetjs_jupyter_widget import UpSetWidget

In [2]:
w = UpSetWidget[str]()
w.theme = 'dark'
w.mode = 'hover'

In [3]:
w.from_list(sets=dict(one = ['a', 'b', 'c', 'e', 'g', 'h', 'k', 'l', 'm'], two = ['a', 'b', 'd', 'e', 'j'], three = ['a', 'e', 'f', 'g', 'h', 'i', 'j', 'l', 'm']))
pass

In [4]:
w.elems

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm']

In [5]:
w.sets

[UpSetSet(name=one, elems={'m', 'g', 'e', 'l', 'a', 'b', 'h', 'k', 'c'}),
 UpSetSet(name=three, elems={'m', 'g', 'i', 'e', 'l', 'a', 'f', 'h', 'j'}),
 UpSetSet(name=two, elems={'a', 'b', 'j', 'd', 'e'})]

In [6]:
w

UpSetWidget(value=None, combinations={'type': 'intersection', 'order': 'cardinality'}, elems=['a', 'b', 'c', '…

In [7]:
def selection_changed(s, ws):
    # w.selection = ws["name"] if ws else None
    return s, ws # s["name"] if s else None
interact(selection_changed, s=10, ws=w)

interactive(children=(IntSlider(value=10, description='s', max=30, min=-10), UpSetWidget(value=None, combinati…

<function __main__.selection_changed(s, ws)>